In [1]:
import dataset 
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import time
import pickle
from pathlib import Path
import math
import re

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from  numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
print(tf.__version__)

1.8.0


load du lieu

In [3]:
# khoi tao gia tri load du lieu


train_path= r"D:\Hoang\data_kata_khoaluan"
train_path_experiment =r"D:\Hoang\data_test"

classes = dataset.get_labels(train_path_experiment)

print(classes)

num_classes = len(classes)
print(num_classes)

['a_h', 'a_k', 'chi_h', 'chi_k', 'e_h', 'e_k', 'i_h', 'i_k']
8


In [4]:
#load du lieu 

num_input_channels = 1
validation_size = 0.1
test_size = 0.2
img_size = 64

class DataCompression(object):
    def __init__(self,data):
        self.train = data.train
        self.valid = data.valid
        self.test = data.test

p=Path(r"D:\Hoang\train_data_experiment.file")
 
if not p.exists():
    print("Get and Compress data")
    data_train=dataset.read_train_sets(train_path=train_path_experiment,image_size=img_size,test_size=test_size,validation_size=validation_size)
    data_train = DataCompression(data_train)
    
    with open(p, "wb") as f:
        pickle.dump(data_train, f, pickle.HIGHEST_PROTOCOL)
    
#read data from file data compression
with open(p,"rb") as f:
    print("uncompress data")
    data = pickle.load(f)
    


uncompress data


In [5]:
print(data.train.images.shape)
print(data.valid.images.shape)
print(data.test.images.shape)

(76995, 64, 64, 1)
(10999, 64, 64, 1)
(21998, 64, 64, 1)


# Xây dựng hàm cho CNN

In [6]:
# cau truc CNN M6-1

# filter_size_conv1 = 3 
# num_filters_conv1 = 32

# filter_size_conv2 = 3
# num_filters_conv2 = 32

# filter_size_conv3 = 3
# num_filters_conv3 = 64

# filter_size_conv4 = 3
# num_filters_conv4 = 64
    
# fc_layer_size = 256
        
num_input_channels = 1

#cau truc CNN 
filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 64

fc_layer_size = 512

In [7]:
def create_weights(shape,name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05),name=name)

def create_biases(size,name):
    return tf.Variable(tf.constant(0.05, shape=[size]),name=name)

In [8]:
def create_conv_layer( input,
               num_input_channels, # so channels của từng ảnh
               filter_size,    # kích thước filter, kenel, 
               num_filters,  # số lượng đầu ra cho layer sau
               use_maxpool=False, #sử dụng max_pooling k?
                 name ="conv_x"    ): 
    with tf.name_scope(name):
        #tao weight và bias cho lop convolutional nay/
        print("input {}: {}".format(name,input))

        name_weights = name+"_weights"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[filter_size, filter_size, num_input_channels, num_filters],name=name_weights)
#         print("weights {}: {}".format(name,weights))

        name_biases = name+"_biases"
        with tf.name_scope("biases"):
            biases = create_biases(num_filters,name=name_biases)
        ## xay dung convolution layer, layer trả về có dimension nhu input
#         with tf.device('/gpu:0'):
        layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
#         print("layer {}: {}".format(name,layer))

        layer += biases
        layer = tf.nn.relu(layer)

        if use_maxpool:
            layer = tf.nn.max_pool(value=layer,
                                    ksize=[1, 2, 2, 1],
                                    strides=[1, 2, 2, 1],
                                    padding='SAME')


#         store in histogram
        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        
        layer = tf.identity(layer,name=name)
        return layer
    
def create_flatten_layer(layer,name="flatten"):
    with tf.name_scope(name):
        # chuyen mỗi ảnh thanh mot vector, moi anh sẽ có kích thước phẳng là: img_size * img_size *num_channels

        #dimension layer: batch_size,img_size,img_size,num_channels 

        #lấy số lượng data
        layer_shape = layer.get_shape()

        # số lượng feature mỗi ảnh = img_size * img_size *num_channels
        num_features = layer_shape[1:4].num_elements()

        # reshape lại data ví dụ (10,2,2,5) // có 10 ảnh và mỗi ảnh có 2*2*5=20 feature => flat_layer: (10,20) 
        layer = tf.reshape(layer, [-1, num_features])
#         print("layer {}: {}".format(name,layer))
        layer = tf.identity(layer,name=name) # set name for layer
    
        return layer,num_features

def create_fc_layer(input,num_inputs,num_outputs,use_relu=False,use_dropout=False,keep_prob=1,name = "fc"):
    
    with tf.name_scope(name):
        name_weights = name+"_weights"
        name_biases = name+"_biases"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[num_inputs, num_outputs],name=name_weights)
        with tf.name_scope("biases"):
            biases = create_biases(num_outputs,name=name_biases)
    
#         with tf.device('/gpu:0'):
        layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer)
        if use_dropout:
             layer=tf.nn.dropout(layer,keep_prob);
#         print("layer {}: {}".format(name,layer))

        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        layer = tf.identity(layer,name=name)
        return layer

# Xây dựng cấu trúc CNN
conv3-32 -> conv3-32 ->maxpool -> conv3-64 -> conv3-64 -> maxpool ->fc-256 dropout->fc10

In [9]:
# conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
#                            filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=False, name="conv1")

# conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
#                           filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

# conv3 = create_conv_layer(input=conv2,num_input_channels=num_filters_conv2,
#                           filter_size=filter_size_conv3,num_filters=num_filters_conv3,use_maxpool=False, name="conv3")

# conv4 = create_conv_layer(input=conv3,num_input_channels=num_filters_conv3,
#                           filter_size=filter_size_conv4,num_filters=num_filters_conv4,use_maxpool=True, name="conv4")

# layer_flat = create_flatten_layer(conv4, use_dropout=True)

# num_feature = layer_flat.get_shape()[1:2].num_elements()

# fc1 = create_fc_layer(input=layer_flat,num_inputs=num_feature,num_outputs=fc_layer_size,use_relu=True,use_dropout=True,name="fc1")


# fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,use_relu=False,use_dropout=False,name="fc2")



In [10]:
## My model conv3-32 ->maxpool -> conv3-64->maxpool -> flatten dropout-> fc512 dropout  ->fc10
def my_model(x,keep_prob):
    conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
                               filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=True, name="conv1")

    conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
                              filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

    layer_flat,num_features = create_flatten_layer(conv2)

    fc1 = create_fc_layer(input=layer_flat,num_inputs=num_features,num_outputs=fc_layer_size,
                          use_relu=True,use_dropout=True,keep_prob=keep_prob,name="fc1")


    fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,
                          use_relu=False,use_dropout=False,keep_prob=keep_prob,name="fc2")



In [11]:
def m16(input_x,keep_prob):
    num_input_channels=1
    filter_3 =3
    num_filter_64 = 64
    num_filter_128 = 128
    num_filter_256 =256
    num_filter_512=512

    
    conv1_num_filters = num_filter_64
    conv1_maxpool=False
    
    conv2_num_filters = num_filter_64
    conv2_maxpool = True
    
    conv3_num_filters = num_filter_128
    conv3_maxpool = False
    
    conv4_num_filters = num_filter_128
    conv4_maxpool = True
    
    conv5_num_filters = num_filter_256
    conv5_maxpool = False
    
    conv6_num_filters = num_filter_256
    conv6_maxpool = False
    
    conv7_num_filters = num_filter_256
    conv7_maxpool = True
    
    conv8_num_filters = num_filter_512
    conv8_maxpool = False
    
    conv9_num_filters = num_filter_512
    conv9_maxpool = False
    
    conv10_num_filters = num_filter_512
    conv10_maxpool = True
    
    conv11_num_filters = num_filter_512
    conv11_maxpool = False
    
    conv12_num_filters = num_filter_512
    conv12_maxpool = False
    
    conv13_num_filters = num_filter_512
    conv13_maxpool = True
    
    
    fc1_size=1024
    fc1_use_dropout=True
#     fc1_keep_prob= 0.5
    
    fc2_size =1024
    fc2_use_dropout=True
#     fc2_keep_prob=0.5
    
    conv1 = create_conv_layer(input=input_x,num_input_channels=num_input_channels,
                           filter_size=filter_3,num_filters=conv1_num_filters,use_maxpool=conv1_maxpool,name="conv1")
    conv2 = create_conv_layer(input=conv1,num_input_channels=conv1_num_filters,
                           filter_size=filter_3,num_filters=conv2_num_filters,use_maxpool=conv2_maxpool,name="conv2")
    
    conv3 = create_conv_layer(input=conv2,num_input_channels=conv2_num_filters,
                           filter_size=filter_3,num_filters=conv3_num_filters,use_maxpool=conv3_maxpool,name="conv3")
    
    conv4 = create_conv_layer(input=conv3,num_input_channels=conv3_num_filters,
                           filter_size=filter_3,num_filters=conv4_num_filters,use_maxpool=conv4_maxpool,name="conv4")
    
    conv5 = create_conv_layer(input=conv4,num_input_channels=conv4_num_filters,
                           filter_size=filter_3,num_filters=conv5_num_filters,use_maxpool=conv5_maxpool,name="conv5")
    
    conv6 = create_conv_layer(input=conv5,num_input_channels=conv5_num_filters,
                           filter_size=filter_3,num_filters=conv6_num_filters,use_maxpool=conv6_maxpool,name="conv6")
    
    conv7 = create_conv_layer(input=conv6,num_input_channels=conv6_num_filters,
                           filter_size=filter_3,num_filters=conv7_num_filters,use_maxpool=conv7_maxpool,name="conv7")
    
    conv8 = create_conv_layer(input=conv7,num_input_channels=conv7_num_filters,
                           filter_size=filter_3,num_filters=conv8_num_filters,use_maxpool=conv8_maxpool,name="conv8")
    
    conv9 = create_conv_layer(input=conv8,num_input_channels=conv8_num_filters,
                           filter_size=filter_3,num_filters=conv9_num_filters,use_maxpool=conv9_maxpool,name="conv9")
    
    conv10 = create_conv_layer(input=conv9,num_input_channels=conv9_num_filters,
                           filter_size=filter_3,num_filters=conv10_num_filters,use_maxpool=conv10_maxpool,name="conv10")
    
    conv11 = create_conv_layer(input=conv10,num_input_channels=conv10_num_filters,
                           filter_size=filter_3,num_filters=conv11_num_filters,use_maxpool=conv11_maxpool,name="conv11")
    
    conv12 = create_conv_layer(input=conv11,num_input_channels=conv11_num_filters,
                           filter_size=filter_3,num_filters=conv12_num_filters,use_maxpool=conv12_maxpool,name="conv12")
    
    flatten_layer, number_features = create_flatten_layer(layer=conv12,name="flatten_layer")
    
    fc1 = create_fc_layer(input=flatten_layer,num_inputs=number_features,num_outputs=fc1_size,
                          use_relu=True,use_dropout=fc1_use_dropout,keep_prob=keep_prob,name="fc1")
    
    fc2 = create_fc_layer(input=fc1,num_inputs=fc1_size,num_outputs=num_classes,
                      use_relu=True,use_dropout=fc2_use_dropout,keep_prob=keep_prob,name="fc2")
    
    return fc2

predict

In [12]:
def train(func_model, num_epochs, batch_size):
    tf.reset_default_graph()

    with tf.name_scope("inputs"):
        x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_input_channels], name='x')
        y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
        print(f"num_class: {num_classes}")
        keep_prob=tf.placeholder(tf.float32,name="keep_prob")
       
        
    fc2=func_model(x,keep_prob)
        
    with tf.name_scope("Train"):
#     with tf.device('/gpu:0'): 
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2,
                                                        labels=y_true)
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(cross_entropy)
        tf.summary.scalar("loss",loss)
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)


    with tf.name_scope("Test"):
        y_pred = tf.nn.softmax(fc2,name='y_pred')
        y_pred_cls = tf.argmax(y_pred, axis=1)
        y_true_cls = tf.argmax(y_true, axis=1)
        correct_prediction = tf.equal(y_pred_cls, y_true_cls)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

        tf.summary.scalar("accuracy",accuracy)

    
    my_config =tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
    my_config.gpu_options.allow_growth = True
    session = tf.Session(config=my_config)

    session.run(tf.global_variables_initializer())

    saver = tf.train.Saver()
    merged =tf.summary.merge_all();
    
     # create path to save session
    str_time = time.ctime();
    str_time = re.sub("[ :]","_",str_time)

    model_name = func_model.__name__
    folder_model_name = model_name+"_"+str_time

    path_train =fr"..\a_model_khoa_luan\{folder_model_name}\graph\train"
    path_valid = fr"..\a_model_khoa_luan\{folder_model_name}\graph\valid"

    train_writer= tf.summary.FileWriter(path_train,session.graph)
    valid_writer=tf.summary.FileWriter(path_valid,session.graph)
    
    
    num_iteration = int(data.train.images.shape[0]/batch_size)

    print_loss_iteration = int(num_iteration /5);
    total_iterations =0
    for epoch in range(num_epochs):
        for i in range(num_iteration):
            print(f"i: {i}")
            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
    #         print(y_true_batch.shape)
            feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: keep_prob}    

            session.run(train_step, feed_dict=feed_dict_train )
            
            if (i % print_loss_iteration == 0 or i==num_iteration-1): 
                x_valid_batch, y_valid_batch, iname_valid_batch,cls_valid_batch = data.train.next_batch(100)
                
                feed_dict_valid = {x: x_valid_batch, y_true: y_valid_batch, keep_prob:1}
                feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: 1}

                valid_loss,valid_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_valid)
 
                train_loss,train_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_train)
                
                total_iterations=total_iterations+print_loss_iteration

                
                print(f"epoch: {epoch} -- iteration: {total_iterations} -- train_loss: {train_loss:.2f} -- train_acc {train_accuracy:.2%} --valid_loss:{valid_loss:.2f} -- valid_acc {valid_accuracy:.2%} -- time: {time.ctime()}")
#                 if i==num_iteration-1:
#                     print(f"save graph -- epoch: {epoch}-- iteration: {i}")
                train_merged=session.run(merged, feed_dict=feed_dict_train)
                valid_merged = session.run(merged, feed_dict=feed_dict_valid)
                train_writer.add_summary(train_merged,total_iterations)
                valid_writer.add_summary(valid_merged,total_iterations)
            
            
        save_path=saver.save(session, r'..\a_model_khoa_luan\model\modelTest.ckpt',global_step=epoch)

def accuracy_test():
    num_interations = int(data.test.images.shape[0]/500)
    
    for iteration in range(num_interations):
        x_batch_test, y_true_batch_test, _, cls_batch_test = data.test.next_batch(500)
        feed_dict_test = {x:x_batch,y_true:y_batch,keep_prob:1.0}
        acc=session.run(accuracy,feed_dict =feed_dict_test)
        print(f"iter: {iteration} -- accuracy test : {acc}")

In [13]:
start_time = time.time()
print(f"start time: {time.ctime(start_time)}")
train(func_model=m16,num_epochs=3, batch_size =50)
end_time= time.time()
print(f"end time: {time.ctime(start_time)}")
run_time =(end_time-start_time)/60
print(f"run time: {run_time} m ")

start time: Sat Aug 11 23:08:38 2018
num_class: 8
input conv1: Tensor("inputs/x:0", shape=(?, 64, 64, 1), dtype=float32)
input conv2: Tensor("conv1/conv1:0", shape=(?, 64, 64, 64), dtype=float32)
input conv3: Tensor("conv2/conv2:0", shape=(?, 32, 32, 64), dtype=float32)
input conv4: Tensor("conv3/conv3:0", shape=(?, 32, 32, 128), dtype=float32)
input conv5: Tensor("conv4/conv4:0", shape=(?, 16, 16, 128), dtype=float32)
input conv6: Tensor("conv5/conv5:0", shape=(?, 16, 16, 256), dtype=float32)
input conv7: Tensor("conv6/conv6:0", shape=(?, 16, 16, 256), dtype=float32)
input conv8: Tensor("conv7/conv7:0", shape=(?, 8, 8, 256), dtype=float32)
input conv9: Tensor("conv8/conv8:0", shape=(?, 8, 8, 512), dtype=float32)
input conv10: Tensor("conv9/conv9:0", shape=(?, 8, 8, 512), dtype=float32)
input conv11: Tensor("conv10/conv10:0", shape=(?, 4, 4, 512), dtype=float32)
input conv12: Tensor("conv11/conv11:0", shape=(?, 4, 4, 512), dtype=float32)
epoch: 0 -- iteration: 307 -- train_loss: 2.16 -

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-733cd0e61921>", line 3, in <module>
    train(func_model=m16,num_epochs=3, batch_size =50)
  File "<ipython-input-12-469cbb0a083d>", line 68, in train
    session.run(train_step, feed_dict=feed_dict_train )
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1135, in _run
    feed_dict_tensor, options, run_metadata)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1316, in _do_run
    run_metadata)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1322, in _do_call
    return fn(*args)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1307, in _

KeyboardInterrupt: 

In [ ]:
# accuracy_test()

In [ ]:
80*64*64/100

In [ ]:
h